In [ ]:
!pip install tqdm

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
phobert = AutoModel.from_pretrained("vinai/phobert-base-v2")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/678 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch_xla/__init__.py:253: UserWarning: `tensorflow` can conflict with `torch-xla`. Prefer `tensorflow-cpu` when using PyTorch/XLA. To silence this warning, `pip uninstall -y tensorflow && pip install tensorflow-cpu`. If you are in a notebook environment such as Colab or Kaggle, restart your notebook runtime afterwards.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/540M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.13M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
class AccentDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text_in = self.data[idx][0]
        text_out = self.data[idx][1]

        encoding_in = self.tokenizer(text_in,
                                     add_special_tokens=True,
                                     max_length=self.max_length,
                                     padding="max_length",
                                     truncation=True,
                                     return_tensors="pt")

        encoding_out = self.tokenizer(text_out,
                                      add_special_tokens=True,
                                      max_length=self.max_length,
                                      padding="max_length",
                                      truncation=True,
                                      return_tensors="pt")

        # Lấy id của từng token trong output (bao gồm cả dấu)
        labels = encoding_out['input_ids'].squeeze()

        return {
            'input_ids': encoding_in['input_ids'].squeeze(),
            'attention_mask': encoding_in['attention_mask'].squeeze(),
            'labels': labels
        }
max_length = 128
batch_size = 2048


In [ ]:
train_data = [
    ("xin chao", "Xin chào"),
    ("toi la Toan", "Tôi là Toàn"),
    ("rat vui duoc gap ban", "Rất vui được gặp bạn"),
    ("hom nay troi dep qua", "Hôm nay trời đẹp quá"),
    ("minh di hoc", "Minh đi học"),
    ("toi thich an kem", "Tôi thích ăn kem"),
    ("ban hoc lop may", "Bạn học lớp mấy"),
    ("co giao rat xinh", "Cô giáo rất xinh"),
    ("toi muon di choi", "Tôi muốn đi chơi"),
    ("ngay mai la chu nhat", "Ngày mai là chủ nhật"),
    ("toi di choi cong vien", "Tôi đi chơi công viên"),
    ("ban thich choi game gi", "Bạn thích chơi game gì"),
    ("toi thich xem phim", "Tôi thích xem phim"),
    ("bo me toi deu la giao vien", "Bố mẹ tôi đều là giáo viên"),
    ("toi co mot con meo", "Tôi có một con mèo"),
    ("nha toi o dau pho", "Nhà tôi ở đầu phố"),
    ("toi thich uong ca phe", "Tôi thích uống cà phê"),
    ("ban di hoc bang xe gi", "Bạn đi học bằng xe gì"),
    ("toi di bo den truong", "Tôi đi bộ đến trường"),
    ("hom nay toi met lam", "Hôm nay tôi mệt lắm"),
    ("ban co khoe khong", "Bạn có khỏe không"),
    ("toi bi om", "Tôi bị ốm"),
    ("toi muon di du lich", "Tôi muốn đi du lịch"),
    ("ban da tung di da lat chua", "Bạn đã từng đi Đà Lạt chưa"),
    ("toi thich bien", "Tôi thích biển"),
    ("mua he rat nong", "Mùa hè rất nóng"),
    ("toi thich mua dong", "Tôi thích mùa đông"),
    ("toi thich an banh chung", "Tôi thích ăn bánh chưng"),
    ("tet nguyen dan sap den roi", "Tết Nguyên Đán sắp đến rồi"),
    ("toi muon nhan duoc nhieu li xi", "Tôi muốn nhận được nhiều lì xì"),
    ("ban co bao nhieu bo quan ao", "Bạn có bao nhiêu bộ quần áo"),
    ("toi thich mau xanh", "Tôi thích màu xanh"),
    ("con so may ban", "Con số mấy bạn"),
    ("toi khong biet", "Tôi không biết"),
    ("ban hay cho toi biet di", "Bạn hãy cho tôi biết đi"),
    ("duoc thoi", "Được thôi"),
    ("cam on ban", "Cảm ơn bạn"),
    ("khong co gi", "Không có gì"),
    ("toi phai di hoc", "Tôi phải đi học"),
    ("hen gap lai ban sau", "Hẹn gặp lại bạn sau"),
    ("chuc ban mot ngay tot dep", "Chúc bạn một ngày tốt đẹp"),
    ("toi cung vay", "Tôi cũng vậy"),
    ("toi rat vui", "Tôi rất vui"),
    ("toi rat buon", "Tôi rất buồn"),
    ("toi dang buon ngu", "Tôi đang buồn ngủ"),
    ("toi muon di ngu", "Tôi muốn đi ngủ"),
    ("toi muon an com", "Tôi muốn ăn cơm"),
    ("toi doi bung qua", "Tôi đói bụng quá"),
    ("mon nay ngon qua", "Món này ngon quá"),
    ("toi thich an do an viet nam", "Tôi thích ăn đồ ăn Việt Nam"),
    ("ban co thich an cay khong", "Bạn có thích ăn cay không"),
    ("toi thich uong tra sua", "Tôi thích uống trà sữa"),
    ("ban co thich xem bong da khong", "Bạn có thích xem bóng đá không"),
    ("toi thich doi tuyen viet nam", "Tôi thích đội tuyển Việt Nam"),
    ("toi thich nghe nhac", "Tôi thích nghe nhạc"),
    ("ban thich ca si nao", "Bạn thích ca sĩ nào"),
    ("toi thich son tung mtp", "Tôi thích Sơn Tùng MTP"),
    ("ban hay hat cho toi nghe mot bai di", "Bạn hãy hát cho tôi nghe một bài đi"),
    ("toi hat khong hay", "Tôi hát không hay"),
    ("khong sao dau", "Không sao đâu"),
    ("cu hat di", "Cứ hát đi"),
    ("ban hat hay qua", "Bạn hát hay quá"),
    ("toi thich doc truyen", "Tôi thích đọc truyện"),
    ("ban thich tac gia nao", "Bạn thích tác giả nào"),
    ("toi thich nguyen nhat anh", "Tôi thích Nguyễn Nhật Ánh"),
    ("ban da doc truyen nay chua", "Bạn đã đọc truyện này chưa"),
    ("truyen nay rat hay", "Truyện này rất hay"),
    ("toi muon mua mot cuon sach", "Tôi muốn mua một cuốn sách"),
    ("ban co sach nao hay gioi thieu khong", "Bạn có sách nào hay giới thiệu không"),
    ("toi thich di hoc o thu vien", "Tôi thích đi học ở thư viện"),
    ("thu vien co rat nhieu sach", "Thư viện có rất nhiều sách"),
    ("toi muon lam viec o google", "Tôi muốn làm việc ở Google"),
    ("ban muon lam nghe gi", "Bạn muốn làm nghề gì"),
    ("toi muon tro thanh bac si", "Tôi muốn trở thành bác sĩ"),
    ("toi muon giup do moi nguoi", "Tôi muốn giúp đỡ mọi người"),
    ("nghe bac si rat cao quy", "Nghề bác sĩ rất cao quý"),
    ("toi muon di du hoc", "Tôi muốn đi du học"),
    ("ban muon di du hoc o dau", "Bạn muốn đi du học ở đâu"),
    ("toi muon di du hoc o my", "Tôi muốn đi du học ở Mỹ"),
    ("tieng anh cua toi chua tot", "Tiếng Anh của tôi chưa tốt"),
    ("toi dang co gang hoc tieng anh", "Tôi đang cố gắng học tiếng Anh"),
    ("ban hoc tieng anh nhu the nao", "Bạn học tiếng Anh như thế nào"),
    ("toi hoc tieng anh qua phim anh", "Tôi học tiếng Anh qua phim ảnh"),
    ("phim anh giup toi cai thien kha nang nghe noi", "Phim ảnh giúp tôi cải thiện khả năng nghe nói"),
    ("toi cung thich xem phim my", "Tôi cũng thích xem phim Mỹ"),
    ("ban co thich xem phim han quoc khong", "Bạn có thích xem phim Hàn Quốc không"),
    ("toi thich xem phim tinh cam", "Tôi thích xem phim tình cảm"),
    ("phim han quoc rat hay", "Phim Hàn Quốc rất hay"),
    ("dien vien dep trai qua", "Diễn viên đẹp trai quá"),
    ("toi cung thay vay", "Tôi cũng thấy vậy"),
    ("ban co nguoi yeu chua", "Bạn có người yêu chưa"),
    ("toi chua co nguoi yeu", "Tôi chưa có người yêu"),
    ("ban muon tim nguoi yeu nhu the nao", "Bạn muốn tìm người yêu như thế nào"),
    ("toi muon tim nguoi yeu hieu thao", "Tôi muốn tìm người yêu hiểu thảo"),
    ("dieu do quan trong nhat", "Điều đó quan trọng nhất"),
    ("ngoai hinh khong quan trong", "Ngoại hình không quan trọng"),
    ("tinh yeu la phai den tu hai phia", "Tình yêu là phải đến từ hai phía"),
    ("toi dong y voi ban", "Tôi đồng ý với bạn"),
    ("ban noi dung lam", "Bạn nói đúng lắm"),
    ("toi thich di bien vao mua he", "Tôi thích đi biển vào mùa hè"),
    ("con ban thi sao", "Còn bạn thì sao"),
    ("toi thich di nui vao mua dong", "Tôi thích đi núi vào mùa đông"),
    ("khung canh rat dep", "Khung cảnh rất đẹp"),
    ("toi thich chup anh", "Tôi thích chụp ảnh"),
    ("ban co may anh chua", "Bạn có máy ảnh chưa"),
    ("toi co mot chiec may anh nikon", "Tôi có một chiếc máy ảnh Nikon"),
    ("anh chup rat dep", "Ảnh chụp rất đẹp"),
    ("ban chup anh co nghe khong", "Bạn chụp ảnh có nghề không"),
    ("toi chup anh chi la so thich thoi", "Tôi chụp ảnh chỉ là sở thích thôi"),
    ("ban co choi facebook khong", "Bạn có chơi Facebook không"),
    ("toi co tai khoan facebook", "Tôi có tài khoản Facebook"),
    ("ban cho minh xin link facebook di", "Bạn cho mình xin link Facebook đi"),
    ("minh ket ban nhe", "Mình kết bạn nhé"),
    ("ok luon", "Ok luôn"),
    ("ban dung dien thoai gi vay", "Bạn dùng điện thoại gì vậy"),
    ("toi dung iphone", "Tôi dùng iPhone"),
    ("dien thoai cua ban bao nhieu tien", "Điện thoại của bạn bao nhiêu tiền"),
    ("toi mua hon chuc trieu", "Tôi mua hơn chục triệu"),
    ("dat do nhi", "Đắt đỏ nhỉ"),
    ("nhung ma dung rat thich", "Nhưng mà dùng rất thích"),
    ("toi thich dung hang apple", "Tôi thích dùng hàng Apple"),
    ("con ban thi sao", "Còn bạn thì sao"),
    ("toi thich dung hang samsung", "Tôi thích dùng hàng Samsung"),
    ("moi nguoi mot so thich", "Mỗi người một sở thích"),
    ("dung roi", "Đúng rồi"),
    ("ban dang o dau vay", "Bạn đang ở đâu vậy"),
    ("toi dang o nha", "Tôi đang ở nhà"),
    ("ban co muon di cafe khong", "Bạn có muốn đi cà phê không"),
    ("toi ban mot chut", "Tôi bận một chút"),
    ("vay thoi gian toi goi lai cho ban sau nhe", "Vậy thôi, giờ tôi gọi lại cho bạn sau nhé"),
    ("ok ban nhe", "Ok bạn nhé"),
    ("bai bai ban", "Bai bai bạn"),
    ("chuc ban mot buoi toi vui ve", "Chúc bạn một buổi tối vui vẻ"),
    ("toi cung chuc ban nhu vay", "Tôi cũng chúc bạn như vậy"),
    ("toi di ngu day", "Tôi đi ngủ đây"),
    ("ngon giac nhe", "Ngon giấc nhé"),
    ("cam on ban", "Cảm ơn bạn"),
    ("bai bai", "Bai bai"),
    ("chao buoi sang", "Chào buổi sáng"),
    ("ban da an sang chua", "Bạn đã ăn sáng chưa"),
    ("toi an roi", "Tôi ăn rồi"),
    ("ban an gi vay", "Bạn ăn gì vậy"),
    ("toi an pho", "Tôi ăn phở"),
    ("ngon khong", "Ngon không"),
    ("rat ngon", "Rất ngon"),
    ("ban muon an gi khong", "Bạn muốn ăn gì không"),
    ("toi khong doi", "Tôi không đói"),
    ("vay thoi minh di lam viec tiep day", "Vậy thôi, mình đi làm việc tiếp đây"),
    ("chuc ban mot ngay lam viec hieu qua", "Chúc bạn một ngày làm việc hiệu quả"),
    ("cam on ban", "Cảm ơn bạn"),
    ("toi cung vay", "Tôi cũng vậy"),
]
train_dataset = AccentDataset(train_data, tokenizer, max_length)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
class PhoBERTForAccentPrediction(torch.nn.Module):
    def __init__(self, model_name, num_classes):
        super(PhoBERTForAccentPrediction, self).__init__()
        self.phobert = AutoModel.from_pretrained(model_name)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(self.phobert.config.hidden_size, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.phobert(input_ids=input_ids, attention_mask=attention_mask)
        # Lấy hidden states từ output của PhoBERT
        hidden_states = outputs.last_hidden_state

        # Loại bỏ padding tokens khỏi hidden states bằng cách dùng attention mask
        active_logits = hidden_states[attention_mask == 1]
        active_logits = self.dropout(active_logits)
        logits = self.classifier(active_logits)

        return logits


In [ ]:
def train_epoch(model, data_loader, optimizer, scheduler=None):
    model = model.train()
    losses = []
    for d in tqdm(data_loader):
        input_ids = d["input_ids"]
        attention_mask = d["attention_mask"]
        targets = d["labels"]

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

        loss = torch.nn.functional.cross_entropy(
                                                  outputs.view(-1, outputs.size(-1)),
                                                  targets[attention_mask == 1].view(-1),
                                                  ignore_index=tokenizer.pad_token_id
                                                  )

        loss.backward()
        optimizer.step()
        if scheduler is not None:
            scheduler.step()
        optimizer.zero_grad()
        losses.append(loss.item())
    return torch.tensor(losses).mean()

In [ ]:
model = PhoBERTForAccentPrediction("vinai/phobert-base-v2", tokenizer.vocab_size)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Some weights of RobertaModel were not initialized from the model checkpoint at vinai/phobert-base-v2 and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
epochs = 20
learning_rate = 2e-2

In [ ]:
for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        train_loss = train_epoch(model, train_dataloader, optimizer)
        print(f'Train loss {train_loss}')

Epoch 1/100


100%|██████████| 1/1 [00:13<00:00, 13.31s/it]


Train loss 10.627620697021484
Epoch 2/100


100%|██████████| 1/1 [00:13<00:00, 13.57s/it]


Train loss 10.582107543945312
Epoch 3/100


...

Train loss 8.751445770263672
Epoch 99/100


100%|██████████| 1/1 [00:13<00:00, 13.84s/it]


Train loss 8.744621276855469
Epoch 100/100


100%|██████████| 1/1 [00:13<00:00, 13.93s/it]

Train loss 8.728797912597656


In [ ]:
def test_model(model, sentence, tokenizer, max_length):
    model.eval()  # Chuyển model sang chế độ đánh giá
    # Mã hóa câu đầu vào
    input_ids = tokenizer(sentence, add_special_tokens=True, max_length=max_length,
                          padding="max_length", truncation=True, return_tensors="pt").input_ids

    # Dự đoán
    with torch.no_grad():
        output = model(input_ids=input_ids, attention_mask=input_ids != tokenizer.pad_token_id)
        # Lấy dự đoán là nhãn có xác suất cao nhất
        _, predicted_ids = torch.max(output, dim=1)

    # Chuyển từ id sang token và nối thành câu hoàn chỉnh
    predicted_tokens = tokenizer.convert_ids_to_tokens(predicted_ids)
    predicted_sentence = " ".join(predicted_tokens)

    return predicted_sentence

# --- Ví dụ sử dụng ---
test_sentence = "toi di hoc"
predicted_sentence = test_model(model, test_sentence, tokenizer, max_length)
print(f"Câu đầu vào: {test_sentence}")
print(f"Câu dự đoán: {predicted_sentence}")

Câu đầu vào: toi di hoc
Câu dự đoán: <s> Tôi đi </s> </s>


In [ ]:
test_sentence = "toi di hoc voi ban than nhat cua toi"
predicted_sentence = test_model(model, test_sentence, tokenizer, max_length)
print(f"Câu đầu vào: {test_sentence}")
print(f"Câu dự đoán: {predicted_sentence}")

Câu đầu vào: toi di hoc voi ban than nhat cua toi
Câu dự đoán: <s> Tôi đi học </s> Bạn </s> </s> </s> </s> </s> </s>


In [ ]:
for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        train_loss = train_epoch(model, train_dataloader, optimizer)
        print(f'Train loss {train_loss}')

Epoch 1/20


100%|██████████| 1/1 [00:13<00:00, 13.77s/it]


Train loss 8.623518943786621
Epoch 2/20


100%|██████████| 1/1 [00:13<00:00, 13.86s/it]


Train loss 8.606202125549316
Epoch 3/20


...

Train loss 8.409326553344727
Epoch 19/20


100%|██████████| 1/1 [00:13<00:00, 13.79s/it]


Train loss 8.405435562133789
Epoch 20/20


100%|██████████| 1/1 [00:13<00:00, 13.58s/it]

Train loss 8.38846492767334


In [ ]:
test_sentence = "toi di hoc"
predicted_sentence = test_model(model, test_sentence, tokenizer, max_length)
print(f"Câu đầu vào: {test_sentence}")
print(f"Câu dự đoán: {predicted_sentence}")

Câu đầu vào: toi di hoc
Câu dự đoán: <s> Tôi đi học </s>


In [ ]:
|test_sentence = "toi di hoc voi ban than nhat cua toi"
predicted_sentence = test_model(model, test_sentence, tokenizer, max_length)
print(f"Câu đầu vào: {test_sentence}")
print(f"Câu dự đoán: {predicted_sentence}")

Câu đầu vào: toi di hoc voi ban than nhat cua toi
Câu dự đoán: <s> Tôi đi học </s> Bạn </s> </s> </s> </s> </s> </s>
